In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import gc
import sys
sys.path.append('./tools')

from load_data import load_split_data, load_set_data, load_test_frames, load_frames_data
from input_distortion import input_distortion,pretraining_input_distortion
from visualization import print_training_frames
from copy import copy

from PretrainingMAE import PretrainingMAE
from MAE import MAE

In [ ]:
# Load data for training
train_seq, val_seq, test_seq = load_split_data()

data_train = load_set_data(train_seq)
data_val = load_set_data(val_seq)

In [ ]:
# Define models for Pretraining
pretraining =  PretrainingMAE(data_train,data_val)
del data_train, data_val

In [ ]:
# Pretraing RGB Channels independently
pretraining.pretrain_red_channel()
pretraining.pretrain_green_channel()
pretraining.pretrain_blue_channel()

In [ ]:
# Pretraing Inverse-Depth Channel
pretraining.pretrain_depth_channel()

In [ ]:
# Pretraing Semantic Channels independently
pretraining.pretrain_gnd_channel()
pretraining.pretrain_obj_channel()
pretraining.pretrain_bld_channel()
pretraining.pretrain_veg_channel()
pretraining.pretrain_sky_channel()

In [ ]:
# Pretraing Semantic AutoEncoder
pretraining.pretrain_shared_semantics()

In [ ]:
#del pretraining

# Load data for training and testing
train_seq, val_seq, test_seq = load_split_data()

data_train = load_set_data(train_seq)
data_val = load_set_data(val_seq)
data_test = load_set_data(test_seq)

In [2]:
# Define Full Model
mae = MAE()

In [ ]:
# Train MultiModal AutoEncoder
mae.train_model(data_train,data_val)
del data_train, data_val

In [ ]:
# Evaluate full MAE
mae.evaluate(data_test,run='20171016-131619')

In [3]:
# Load data for testing by frames
test_frames = load_test_frames()

data_test_frames = load_frames_data(test_frames)

In [ ]:
imr_test = []

for j in data_test_frames:
    imr_test.append(j['xcr']/255.)
    
print imr_test

In [4]:
# Evaluate full MAE
mae.evaluate_per_frame(data_test_frames,run='20171016-131619')

INFO:tensorflow:Restoring parameters from models/full/20171016-131619/fullmodel.ckpt


INFO:tensorflow:Restoring parameters from models/full/20171016-131619/fullmodel.ckpt


('Size of Test Set:', 697)
('Error (RMS):', 11.006707114244335)
('Error (Relative Error):', 0.33741376670909767)


In [ ]:
print len(data_test_frames)

In [ ]:
imr = np.reshape(data_test_frames[-1]['xcr']/255.,(60,18)).T
img = np.reshape(data_test_frames[-1]['xcg']/255.,(60,18)).T
imb = np.reshape(data_test_frames[-1]['xcb']/255.,(60,18)).T
im = np.dstack((imr,img,imb))

imd = np.reshape(data_test_frames[-1]['xid'],(60,18)).T * np.reshape(data_test_frames[1]['xmask'],(60,18)).T

ims = np.reshape(data_test_frames[-1]['sem'],(60,18)).T

%matplotlib inline

fig,axes = plt.subplots(3,1)
axes[0].imshow(im)
axes[0].set_title('Image')

axes[1].imshow(imd)
axes[1].set_title('Inverse Depth')

axes[2].imshow(ims)
axes[2].set_title('Semantics')

plt.show()

In [ ]:
#a = np.reshape(pretraining.imr_val[0],(60,18))
#print a[1]
#np.reshape(pretraining.imr_val)
print np.dstack((imr,img,imb)).shape